# Gemini API: Basic information extraction

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/examples/prompting/Basic_Information_Extraction.ipynb"><img src = "https://www.tensorflow.org/images/colab_logo_32px.png"/>Run in Google Colab</a>
  </td>
</table>

This example notebook shows how Gemini API's Python SDK can be used to extract information from a block of text and return it in defined structure.

We want to give the LLM a recipe and extract all the ingredients to create a shopping list.
According to best practices, complex tasks will be executed better if divided into separate steps, such as:

1. First, the model will extract all the groceries into a list.

2. Then, we will prompt it to convert this list into a shopping list.

You can find more tips for writing prompts [here](https://ai.google.dev/gemini-api/docs/prompting-intro).


In [ ]:
!pip install -U -q google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 679.1/679.1 kB 26.5 MB/s eta 0:00:00


In [ ]:
import google.generativeai as genai

from IPython.display import Markdown

## Configure your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## Example

In [ ]:
model = genai.GenerativeModel(model_name='gemini-pro', generation_config={"temperature": 0})

In [ ]:
recipe = """
Step1:
Grind 3 garlic cloves, knob of fresh ginger, roughly chopped, 3 spring onions to a paste in a food processor.
Add 2 tbsp of clear honey, juice from one orange, 1 tbsp of light soy sauce and 2 tbsp of vegetable oil, then blend again.
Pour the mixture over the cubed chicken from 4 small breast fillets and leave to marnate for at least 1hr.
Toss in the 20 button mushrooms for the last half an hour so the take on some of the flavour, too.

Step2:
Thread the chicken, 20 cherry tomatoes, mushrooms and 2 large red peppers onto 20 wooden skewers,
then cook on a griddle pan for 7-8 mins each side or until the chicken is thoroughly cooked and golden brown.
Turn the kebabs frequently and baste with the marinade from time to time until evenly cooked.
Arrange on a platter, scatter with chopped spring onion and eat with your fingers.
"""

In [ ]:
# Step 1 - extract all groceries
groceries = f"""
Your task is to extract to a list all the groceries with its quantities based on the provided recipe: {recipe}.
Make sure that groceries are in the order of appearance.
"""
grocery_list = model.generate_content(groceries)
print(grocery_list.text)

- 3 garlic cloves
- knob of fresh ginger
- 3 spring onions
- 2 tbsp of clear honey
- juice from one orange
- 1 tbsp of light soy sauce
- 2 tbsp of vegetable oil
- 4 small breast fillets
- 20 button mushrooms
- 20 cherry tomatoes
- 2 large red peppers
- 20 wooden skewers


In [ ]:
# Step 2 - format as shopping list
shopping_list = f"""
You are given a list of groceries. Complete the following:
- Organize groceries into categories for easier shopping.
- List each item one under another with a checkbox [].

Example:
List: 3 tomatoes, 1 bread, 4 tomatoes
- [ ] 7 tomatoes
- [ ] 1 Bread

List of groceries: {grocery_list}

"""
Markdown(model.generate_content(shopping_list).text)

**Produce:**
- [ ] 3 garlic cloves
- [ ] knob of fresh ginger
- [ ] 3 spring onions
- [ ] 20 button mushrooms
- [ ] 20 cherry tomatoes
- [ ] 2 large red peppers

**Meat:**
- [ ] 4 small breast fillets

**Condiments:**
- [ ] 2 tbsp of clear honey
- [ ] juice from one orange
- [ ] 1 tbsp of light soy sauce

**Cooking Essentials:**
- [ ] 2 tbsp of vegetable oil
- [ ] 20 wooden skewers

## Next steps

Be sure to explore other examples of prompting in the repository. Try creating your own prompts for information extraction or adapt the ones provided in the notebook.